In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('Human trafficking 15-20.csv')

# Get the list of states from the first column
states = df.iloc[:, 0].unique()

# Create a dictionary to store the models and predictions for each state
models = {}
predictions = {}

# Loop over each state and train a separate SARIMA model
for state in states:
    # Get the data for the current state
    data = df[df.iloc[:, 0] == state].iloc[:, 1:].squeeze()

    # Split the data into training and testing sets
    train_data = data[:-2]
    test_data = data[-2:]

    # Train the SARIMA model
    model = SARIMAX(train_data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit()

    # Make predictions for the next 5 years
    predictions[state] = model_fit.forecast(steps=5)

    # Store the trained model
    models[state] = model_fit

# Plot the predicted values for each state
for state in states:
    # Get the actual and predicted values for the current state
    actual_values = df[df.iloc[:, 0] == state].iloc[:, 1:].squeeze().values
    predicted_values = np.concatenate([actual_values[:-2], predictions[state], np.repeat(predictions[state][-1], 2)])

    # Create a plot for the current state
    plt.figure(figsize=(8, 4))
    plt.plot(actual_values, label='Actual Values')
    plt.plot(predicted_values, label='Predicted Values')
    plt.xticks(np.arange(len(actual_values)), np.arange(2015, 2023), rotation=45)
    plt.xlabel('Year')
    plt.ylabel('Number of Cases')
    plt.title(state)
    plt.legend()
    plt.show()